In [2]:
import torch
import torch.nn as nn
import torchvision.models as models

from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]

IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 2

In [4]:
test_transforms = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
])

test_dataset_path = "../../../dataset/test"
test_dataset = datasets.ImageFolder(root=test_dataset_path, transform=test_transforms)

test_loader = DataLoader(
    test_dataset,
    shuffle=False,
    pin_memory=True,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

In [5]:
def test_model(model, loader, criterion, device):
    model.eval()
    test_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = test_loss / total
    avg_accuracy = correct / total

    return avg_loss, avg_accuracy

In [6]:
model = models.resnet152(weights=None)
trained_model_path = "best_resnet152.pth"

model.load_state_dict(torch.load(trained_model_path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()

In [7]:
test_loss, test_accuracy = test_model(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.3616, Test Accuracy: 0.8794


In [8]:
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
        all_labels.extend(labels.numpy())

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n",
      classification_report(all_labels, all_preds, target_names=test_dataset.classes))

Confusion Matrix:
 [[155   0   8   1   1   5   0   0   0]
 [  0 119  14   0   0   1   0   0   0]
 [  3  30 127   2   0   1   0   0   0]
 [  0   0   2 144   0   0   1   2   1]
 [  1   5   0  13 125   2   0   4   0]
 [  1   0   1   0   0  21   0   0   1]
 [  0   0   0   0   0   0  20   1   0]
 [  2   9   0   5   2   1   0 153   0]
 [  0   0   0   0   0   0   0   0  11]]

Classification Report:
                      precision    recall  f1-score   support

         algal_spot       0.96      0.91      0.93       170
       brown_blight       0.73      0.89      0.80       134
        gray_blight       0.84      0.78      0.81       163
            healthy       0.87      0.96      0.91       150
         helopeltis       0.98      0.83      0.90       150
           red-rust       0.68      0.88      0.76        24
red-spider-infested       0.95      0.95      0.95        21
           red_spot       0.96      0.89      0.92       172
         white-spot       0.85      1.00      0.92    